
## SPARSE DISTRIBUTED MEMORY

I wrote these programs to explore Pentti Kanerva's idea of Sparse Distributed Memory, introduced
in his 1988 book of that title. The software accompanies the article ["The Mind Wanders,"](http://bit-player.org/2018/the-mind-wanders) published in July 2018 on [bit-player.org](http://bit-player.org).

Copyright 2018 Brian Hayes

Released under the [MIT LICENSE](https://opensource.org/licenses/MIT).

The code was written for [Julia](https://julialang.org/) version 0.6.3. Versions 0.7 and 1.0 have now been released. They differ significantly from the earlier code, so that building a dual-use file is not practical. If you're running Julia 0.7 or later, please use sdmj7.ipynb.

Disclaimer: I am a student of Julia, not a master of it. Suggestions and corrections welcome.


### GETTING STARTED

You'll need a working version of the [Julia](https://julialang.org/) programming language, version 0.6, and the [Jupyter](http://jupyter.org/) environment for exploratory programming. And you need to get Julia and Jupyter talking to each other, which entails installing the IJulia package in Julia.

Once you have Julia running in a Jupyter notebook, the next step is to install a couple of needed packages. Here's the incantation:

```
Pkg.add(DataStructures, StatsBase)
```

Now open this file in Jupyter, and evaluate all the cells (_Cell_ menu in Jupyter notebook, _Run_ menu in JupyterLab). 

In a fresh notebook cell (or in a console in JupyterLab), evaluate:

```
init_SDM()
fill_memory(10000)
```
You may see some "Info" messages, but there should not be any Warnings or Errors. On my laptop in Julia 0.6.3, creating the 10,000 entries in the memory takes almost an hour. If you're in a hurry, `fill_memory(1000)` produces a usable system in one-tenth the time, although parameters such as the critical radius for recall will have different values.

Now you're ready to play with the sparse distributed memory. See the section below "Experiments with the SDM" for some ideas. 



### IMPLEMENTATION NOTES

Sparse distributed memory is all about computing with high-dimensional binary
vectors -- long sequences of 0s and 1s interpreted as the coordinates of the
vertices of a hypercube in a high-dimensional space. In the examples considered
here the length of the vectors is generally 1,000 bits, and so there are $2^{1000}$ 
possible vectors, or patterns.

The data structure I have chosen for representing these patterns is the Julia `BitVector`, an
array of single bits, packed into 64-bit words. The standard Julia input and
output routines interpret these bits as Boolean values (`false` and `true`), but I
promise it's safe to think of them as 0s and 1s. A big advantage of using
`BitVectors` is that primitive bitwise operations are executed in parallel on
all the bits of a machine word. On a 64-bit processor,
`xor`-ing two 1,000-bit vectors requires only 16 operations instead of 1,000.
(As it happens, `xor` is the operation we're most concerned with.)

A few alternatives to `BitVector`s are worth considering. My first thought when starting the project was to
use `BigInt`s -- arbitrary-length integers. Here too we get the 64-fold
speedup for bitwise operations such as `xor`. But some necessary operations
are difficult to implement on `BigInt`s, such as the majority-rule algorithm for bit-by-bit
pooling of vectors. _(See below under `registers`.)_ And converting `BigInt`s to other formats is costly. 

Another possible data structure for the binary patterns is an array of integers 
with values of +1 and -1, rather than 1 and 0. This choice would simplify the 
majority-rule operation. However, we lose the ability to operate on 64-bit chunks; 
instead we have to loop over the array elements one by one. This cost makes the overall program
considerably slower. But we'll be using the +1/-1 scheme anyway for a slightly different purpose. 
_(Again, see `registers` below.)_


### PACKAGES, IMPORTS, AND OVERRIDES

In [ ]:

using StatsBase.sample  # for sampling without replacement
using JLD               # storing and loading Julia-flavored HDF5 files
using DataStructures    # for accumulator/counter in neighbor distributions


# DEV TOOLS  -- uncomment if needed

# using BenchmarkTools    # may also require the Compat package
# using Base.Profile      # comment out when not needed


### DISPLAY OF BITVECTORS

# The default display of BitVectors is a list of true and false values,
# which wastes space and is nearly incomprehensible for 1,000-bit vectors.
# Here we pick up a routine called `bitshow` from the Julia source code. 
# When we override the built-in "show" and "print" methods, BitVectors 
# appear as strings of 0s and 1s grouped into 8-bit bytes and 64-bit words.

import Base.show      # so that we can modify display of BitVectors
import Base.print     # ditto

show(x::BitVector) = Base.bitshow(x)
print(x::BitVector) = Base.bitshow(x)


### UTILITY FUNCTIONS

In [ ]:

# Sum of n choose k for k in 1:m. Needed to calculate the 
# number of vectors inside a given Hamming radius.

function cum_binomial(n, m)
    sum = zero(BigInt)
    for k in 1:m
        sum = binomial(BigInt(n), k)
    end
    return sum
end


In [ ]:

# Predicate. Applied to any iterable (such as an array), returns
# `true` iff all elements of the iterable are `==`. (Seems like
# there ought to be an easier way... )

function allequal(iterable)
    length(iterable) < 2 && return true
    val = iterable[1]
    all(x->x==val, iterable)
end


### GLOBALS

In [ ]:

# A parameters turn up repeatedly in the SDM model, and it's important they
# they have consistent values throughout. In particular, all of the pattern
# vectors must have the same dimensionality d. To ensure consistency I have
# made d a global constant, and I've treated N (the number of hard locations)
# and r (the radius of the access circle) in the same way.

# The constant SDM holds a pointer to the memory itself, an array of structures
# that represent Kanerva's hard locations. IDX is an index to the items stored
# memory; for more about this components see the section on "Labels and the IDX."

# A further note on Julia constants: They are not really constant. The compiler 
# will not stop you from assigning a new value to d, as in d = 10000, although 
# it will issue a warning message. The only total no-no is trying to change the
# type of d, as in d = 1000.0. However, even though you can change the value of
# a constant, you can't just treat it as a typed variable. Functions compiled
# when the old value was in effect will likely have that old value baked in. You'll
# need to restart the Julia system to have the new value recognized.

# So why not just make the parameters global variables rather than constants? 
# Efficiency. Julia's type inference system can't trust anything about a global
# variable; its type could change from moment to moment, and so it has to be
# checked at runtime.



const N = 10^6    # number of hard locations in the sparse distributed memory
const d = 1000    # dimensions of vectors and registers
const r = 451     # radius of the access circle for sparse distributed memory

struct Loc                      # each hard location has an address and
    address::BitVector          # a register for pooled content
    register::Vector{Int8}
end

const SDM = Vector{Loc}(N)      # the N-element vector of hard locations

const IDX = Dict{String, BitVector}()    # for keeping track of what we've stored


### BINARY VECTORS

The basic unit of information in the SDM is a $d$-dimensional vector of bits. Mostly,
we want _random_ vectors. Randomness assures that all the vectors will,
with high probability, be widely separated in the $d$-dimensional space.


In [ ]:
### MAKING BITVECTORS

make_rand_bitvec() = bitrand(d)

make_zeros_bitvec() = falses(d)

make_ones_bitvec() = trues(d)


# The functions above create fresh vectors suitable for modification.
# If we're just using a vector as a target of comparison with some other
# vector, we can save outselves the trouble of building a fresh copy
# and just use a constant.

const zeros_bitvec = falses(d)
const ones_bitvec  = trues(d);


In [ ]:
### DISTANCE BETWEEN BITVECTORS

# We measure the distance between two vectors in terms of Hamming distance,
# or in other words by the Manhattan metric. For binary vectors this is just
# the number of bit positions at which two vectors differ. We can calculate
# this value by xor-ing the two vectors and counting the 1s in the result.
# (Note that the dot in "xor.(u, v)" signifies that we are broadcasting the
# xor operation elementwise across the two vectors.)

# An important observation is that for large d the distance between any two
# random d-bit vectors will almost surely be close to d/2.

hamming_distance(u::BitVector, v::BitVector) = count(xor.(u, v))

# The only other meaningful form of comparison between binary patterns in the SDM is
# simple equality, which can be checked by the standard operator `==`. There's
# no sense of ranking or comparing magnitudes. No BitVector is greater than or
# less than another.


In [ ]:
### STATISTICS OF NEIGHBOR DISTANCES

# Generate 'reps' pairs of random vectors, and build a histogram of the Hamming
# distances between them.

function distance_histogram(reps)
    ct = counter(Int64)
    for i = 1:reps
        dist = hamming_distance(make_rand_bitvec(), make_rand_bitvec())
        push!(ct, dist)
    end
    return sort(collect(ct), by = tuple -> first(tuple))
end


In [ ]:
### PARSE PRINTED REPRESENTATION OF BITVECTOR

# Sometimes you might want to take the output of show() or print()
# and feed it back into the program.

function parse_vector_rep(s::String)
    v = BitVector()
    for c in s
        if c == '1'
            push!(v, true)
        elseif c == '0'
            push!(v, false)
        end
    end
    return v
end

### REGISTERS

Apart from the binary vectors, there's another essential data structure, which I'm calling 
a `register`. It too is a$d$-dimensional vector, but the components are not individual bits 
but rather 8-bit signed integers (Julia type `Int8`). In the SDM, a register holds the result 
of "pooling" multiple `BitVectors`, a process akin to summation or averaging.

Note: An `Int8` has a range of `[-128, +127]`, which means that pooling more than
127 vectors could cause an overflow. Overflow in Julia is particularly pernicious, because
it is treated as _rollover_: `Int8(127) + Int8(1) = -128`. This risk was acceptable in
my experiments because I was never pooling more than about a dozen vectors. If the risk
of rollover worries you, you could change the type to `Int16` at the cost of another
gigabyte of memory.


In [ ]:

# A register is an array of d Int8s, all initialized to zero. 

make_zero_register() = zeros(Int8, d)


# Inject vector v into the pooled contents of register reg. For each 1 bit in
# v the corresponding element of reg is incremented; for each 0 bit the reg
# element is decremented. Thus at any moment the count stored in a register element
# will represent the number of 1s minus the number of 0s received. Note
# that the reg argument is altered, and no value is returned.

function write_to_register!(reg, v::BitVector)
    (length(reg) == length(v)) || throw(DimensionMismatch("Register and vector must have the same length."))
    for i in 1:length(v)
        @inbounds reg[i] += v[i] ? one(Int8) : -one(Int8)
    end
end


# Now for the inverse of write-to-register!. When reading
# from a register, we need to convert the contents back into a BitVector
# representation. Specifically, we take an array of Ints, and convert each
# element of the array into a binary digit. A positive Int becomes a 1, a
# negative Int becomes a 0; a 0 Int is randomly assigned a value of either 1 or 0.

function read_from_register(reg)
    d = length(reg)
    v = make_zeros_bitvec()
    @inbounds for i in 1:d
        if reg[i] > 0
            v[i] = true
        elseif reg[i] == 0
            v[i] = rand(Bool)
        end
    end
    v
end


### DATA STRUCTURES FOR SPARSE DISTRIBUTED MEMORY

So far we've constructed the basic components of an SDM: high-dimensional bit vectors, and registers for pooling them. Now we need a way to store large numbers of vectors, and thereafter retrieve them.

Providing space for all $2^{1000}$ possible vectors is out of the question. In the memory scheme devised by Kanerva, storage is provided in a small, random subset of the total address space; these selected addresses are called _hard locations_. In the reference model there are $10^{6}$ hard locations. The data structure for a hard location gangs together a `BitVector` that serves as an address, and a `register` that will store everything written to that address, by pooling bits into `Int8`s.


In [ ]:
### INITIALIZE THE MEMORY

# Set up a memory with N hard locations, each of which has an address vector
# and a register of d elements. No value is returned; the created data structure
# is bound to the global variable SDM. Running time is a few seconds.

# This is still an empty memory; there is nothing stored in it.

function init_SDM()
    for i in 1:N
        SDM[i] = Loc(make_rand_bitvec(), make_zero_register())
    end
end


In [ ]:
### STORING AND FETCHING

# A conventional computer memory stores each data item at a single address, and
# each such address can hold only one data item. The SDM is quite different. A data
# item is written at many addresses -- perhaps 1,000 -- and each address holds 
# multiple items, all mixed together.


# How to write a data vector `v` into the SDM: Scan the entire array of hard locations,
# measuring the hamming distance between `v` and each location's address; whenever the
# distance is less than or equal to the access radius `r`, write the vector into the location's
# register. With a million hard locations and `r == 451`, there are typically about 1,000
# locations inside the circle of radius `r`.

function SDMstore(v::BitVector)
    for loc in SDM
        if hamming_distance(v, loc.address) <= r
            write_to_register!(loc.register, v)
         end
     end
end


# Reading from the SDM is also a distributed process. When you ask for the content of
# the memory at address `v`, the `SDMfetch` procedure searches for all hard locations
# within `r` bits of `v`, and pools their contents. The value returned is computed
# by a bit-by-bit majority-rule algorithm.

# `SDMstore(v)` followed by `SDMfetch(v)` will return the value `v` with high probability
# (near certainty if the memory is not filled beyond capacity). If a vector `u` has NOT
# been stored in the memory, `SDMfetch(u)` returns an essentially random result.

function SDMfetch(v::BitVector)
    accum = zeros(Int64, length(v))
    for loc in SDM
        if hamming_distance(v, loc.address) <= r
            accum += loc.register
        end
     end
     return read_from_register(accum)
 end


# SDMsearch attempts to retrieve a stored item `v` based on an approximate cue `u`. The
# attempt is likely to succeed if the distance between `u` and `v` is not too great. (The
# meaning of "not too great" depends on d, r, N, and the number of stored items M.) The
# algorithm is simply a recursive application of SDMfetch, which in successful cases
# will converge on a fixed point. The parameter `limit=30` controls how long to keep
# trying. 

function SDMsearch(u::BitVector, limit=30)
    prev = copy(u)
    for i in 1:limit
        next = SDMfetch(prev)
        if next == prev
            return next
        else
            prev = next
        end
    end
    return zeros_bitvec    # sentinel indicating failure
end




# To test SDMsearch, we need to be able to displace a vector by a specified
# distance, but in a random direction. Thus we have:
#
#    hamming_distance(v, displace(v, 200)) == 200

function displace(v, distance)
    w = copy(v)
    targets = sample(collect(1:d), distance, replace=false, ordered=true)
    for i in targets
        w[i] = ~v[i]
    end
    return w
end


### LABELS AND THE IDX

The set of functions introduced so far amounts to a full memory system: You can store things in it and get them back again, even if you have only a partial address. As a model of biological memory, it could be considered complete. But it's also very hard for the human observer to follow what's going on. A long sequence of 1s and 0s might represent a remembered concept such as "sage" or "butterfly," but on a computer screen such patterns are incomprehensible; we cannot associate them with their meanings. 

To deal with this problem, we can attach a label to each pattern. The labels are short character strings that are easier to recognize than long vectors. But keep in mind: The labels are metadata, outside the memory system itself. They could be removed entirely and nothing would change in the operation of the SDM.

When a pattern is committed to memory, it goes into a dictionary, `IDX`, where the label is the key and the pattern is the associated value. The label is any valid character string. The stored binary vector is generated `make_rand_bitvec()`.

Even apart from the labels, it's important to keep a list of everything that's been stored in the memory. Without it, there's no practical way of ever getting anything out again.


In [ ]:

# To implement the labeling system, we add new methods to several functions. In most
# cases the new methods accept a label in the place of a BitVector.


# Create an IDX entry where `(key, value) == (label, v)`. If v is not supplied,
# generate a random BitVector of length d. Finally, store v in the SDM. Note that
# if `label` is already present in IDX, then any supplied value of v will be
# ignore, and the existing vector from IDX will be stored instead. Thus a value
# cannot be overwritten in IDX, but the same value can be stored multiple times
# in SDM.

function SDMstore(label::String, v::BitVector = make_rand_bitvec())
    w = get!(IDX, label, v)
    SDMstore(w)
end


# Retrieve an item from SDM based on its identifying label in IDX.

function SDMfetch(label::String)
    haskey(IDX, label) || error("Item $label not indexed.")
    SDMfetch(IDX[label])
end


# Do a recursive search of SDM for the vector associated with `label`.

function SDMsearch(label::String, limit=30)
    haskey(IDX, label) || error("Item $label not indexed.")
    SDMsearch(IDX[label], limit)
end
    

# Inverse access to IDX: Given a stored vector, sequentially search IDX
# and return the corresponding label. This allows for usage like so:
#
#      IDXlookup(SDMfetch("beluga")) --> "beluga"

function IDXlookup(v::BitVector)
    for (label, vec) in IDX
        if v == vec
            return label
        end
    end
    return ""
end


# Look up the vector associated with `label`, and return a new vector
# displaced by `distance`.

function displace(label::String, distance)
    haskey(IDX, label) || error("Item $label not indexed.")
    displace(IDX[label], distance)
end


# Another convenience function: Calculate distance between vectors
# given their labels.

function hamming_distance(ulabel::String, vlabel::String)
    haskey(IDX, ulabel) || error("Item $ulabel not indexed.")
    haskey(IDX, vlabel) || error("Item $vlabel not indexed.")
    hamming_distance(IDX[ulabel], IDX[vlabel])
end


# Meaningful labels are great, but I don't want to have to come up with
# 10,000 of them in order to fill up the memory for testing purposes.
# Instead, let's do it with random labels. (Julia's `randstring` draws
# characters from an alphabet of size 62; the chance of a repeated
# eight-character label is about $10^{-14}$)

random_label(n=8) = randstring(n)


# Add C randomly labeled random vectors to the IDX, and store them in 
# the SDM. This is a lengthy operation. On my laptop, `fill_memory(10000)`
# takes almost an hour.

function fill_memory(C)
    for i in 1:C
        SDMstore(random_label())
    end
end


# Finally, we may need to clear the slate.

function init_IDX()
    for k in keys(IDX)
        delete!(IDX, k)
    end
end





### SAVING THE MODEL TO DISK

The data structures for the SDM and the IDX are serialized and saved in the JLD file format, a Julia-adapted flavor of HDF5. For more on what's going on here, see [the Julia Data module](https://github.com/JuliaIO/JLD.jl).

In [ ]:

# Write it to disk...

function save_memory(filename::String)
    if length(filename) < 5 || filename[end-3:end] != ".jld"     # ensure .jld extension
        filename = filename * ".jld"
    end
    save(filename, "SDM", SDM, "IDX", IDX)
end


# ... and read it back in.

function load_memory(filename::String)
    return (load(filename, "SDM"), load(filename, "IDX"))
end


# The file created will be about 2.7 gigabytes. Saving it takes roughly
# 15 minutes, a little less to load it.



# The `load_memory` function should be invoked in the following form:
#
# (SDM, IDX) = load_memory(filename)
#
# You will see two warnings noting the redefinition of the
# constants SDM and IDX. 



### EXPERIMENTS WITH THE SDM

We now have a sparse distributed memory stuffed with 10,000 randomly labeled random patterns. It's time to test retrieving those patterns, and then see what else it might be fun to try.

In [ ]:

# Step through the entire index of 10,000 labels, fetch each of the corresponding
# vectors from the SDM, and make sure it matches the one in the IDX. There should
# be no failures reported.

function test_exact_recall()
    failures = []
    failure_count = 0
    for label in keys(IDX)
        v = IDX[label]
        w = SDMfetch(v)
        if w != v
            push!(failures, label)
            failure_count += 1
        end
    end
    return (failure_count, failures)
end


# Choose a stored vector at random, displace it in a random direction
# by `distance`, and see whether this approximate probe retrieves the
# correct v from the SDM. Repeat `reps` times. The outcome will depend on 
# the value of `distance`. Below the critical convergence radius (somewhere
# between 200 and 210 for a memory with 10,000 stored elements), failures
# should be few. Above that radius, successes will be few. In one experiment
# at distance=200, I saw 43/100 failures.

function test_approx_recall(distance, reps)
    failures = []
    failure_count = 0
    labels = collect(keys(IDX))
    for i in 1:reps
        label = rand(labels)
        v = IDX[label]
        x = displace(v, distance)
        w = SDMsearch(x)
        if w != v
            push!(failures, label)
            failure_count += 1
        end
    end
    return (failure_count, failures)
end


# A more intimate glimpse of the process of converging from an inexact probe.
# For a converging case, the output looks like this:

# start-to-target: 200
# next-prev: 150,  next-start: 150,  next-target: 154
# next-prev: 61,  next-start: 191,  next-target: 129
# next-prev: 25,  next-start: 210,  next-target: 108
# next-prev: 31,  next-start: 217,  next-target: 83
# next-prev: 26,  next-start: 215,  next-target: 59
# next-prev: 38,  next-start: 201,  next-target: 21
# next-prev: 21,  next-start: 200,  next-target: 0
# next-prev: 0,  next-start: 200,  next-target: 0

function trace_recursive_fetch(label, distance, limit=30)
    target = IDX[label]
    start = displace(target, distance)
    prev = start
    println("start-to-target: $(hamming_distance(start, target))")
    for i in 1:limit
        next = SDMfetch(prev)
        np = hamming_distance(next, prev)
        ns = hamming_distance(next, start)
        nt = hamming_distance(next, target)
        println("next-prev: $np,  next-start: $ns,  next-target: $nt")
        prev = next
        np == 0 && break
    end
end



In [ ]:

# List successive values of hamming distance from probe to target
# as the recursive fetching proceeds. E.g.,
#
# show(convergence_trajectory(200, 30)) --> [200, 177, 167, 149, 130, 113, 98, 82, 60, 30, 2, 0]
    
function convergence_trajectory(distance, limit)
    target = rand(collect(values(IDX)))
    probe = displace(target, distance)
    trajectory = Array{Int64, 1}()
    for i in 1:limit
        s = hamming_distance(probe, target)
        push!(trajectory, s)
        if s == 0
            break
        end
        probe = SDMfetch(probe)
    end
    return trajectory
end


In [ ]:

# Tabulate the number of vectors that converge throughout
# a range of displacement values. At each displacement in the range,
# test `reps` randomly chosen vectors.

function convergence_stats(lo, step, hi, reps)
    target = rand(collect(values(IDX)))
    counts = zeros(Int64, hi)
    for i in lo:step:hi
        for j in 1:reps
            probe = displace(target, i)
            distance = i
            limit = 30
            while distance > 0 && limit > 0
                probe = SDMfetch(probe)
                distance = hamming_distance(probe, target)
                limit -= 1
            end
            if distance == 0
                counts[i] += 1
            end
        end
        @printf("%d  %d\n", i, counts[i])
    end
    return counts
end


In [ ]:

# Run `reps` iterations of SDMsearch, all displaced from their
# target by a fixed `distance`, and calculate the fraction that
# converge to the target. 

function convergence_fraction(distance, reps)
    vs = collect(values(IDX))
    hits = 0
    for i in 1:reps
        v = displace(rand(vs), distance)
        if SDMsearch(v) != zeros_bitvec
            hits += 1
        end
    end
    return hits / reps
end
        

# A version with a specific target vector, rather than a randomly
# chosen one.

function convergence_fraction(label::String, distance, reps)
    u = IDX[label]
    hits = 0
    for i in 1:reps
        v = displace(u, distance)
        if SDMsearch(v) != zeros_bitvec
            hits += 1
        end
    end
    return hits / reps
end
        
    
# Apply a bisection method to find the convergence radius -- the
# displacement where half the trials converge and half don't.

function estimate_convergence_radius(lo, hi, reps)
    hi - lo <= 1 && return (lo, hi)   
    mid = div(lo + hi, 2)
    frac = convergence_fraction(mid, reps)
    println("At distance $mid convergence fraction is $frac")
    if frac < 0.5
        estimate_convergence_radius(lo, mid, reps)
    else
        estimate_convergence_radius(mid, hi, reps)
    end
end


# So far, we have been detecting convergence by detecting the presence of
# a fixed point, where SDMfetch(x) = x. But how do we know that the fixed
# point is in fact the target vector we were trying to recall? This procedure
# checks for the possibility of reach other fixed points.

function verify_convergence_destination(ulabel::String, displacement, reps, limit=30)
    converge_count = 0
    failure_count = 0
    wrong_target_count = 0
    for i in 1:reps
        u = IDX[ulabel]
        v = displace(ulabel, displacement)
        w = SDMsearch(v, limit)
        if w == zeros_bitvec
            failure_count += 1
        elseif u == w
            converge_count += 1
        else
            wrong_target_count += 1
            wlabel = IDXlookup(w)
            if wlabel == ""
                wlabel = "Unstored vector"
            end
            uw = hamming_distance(u, w)
            uv = hamming_distance(u, v)
            vw = hamming_distance(v, w)
            println("$i: displace($ulabel, $displacement) --> $wlabel.")
            println("$ulabel = u = ")
            show(u)
            println()
            println("displace($ulabel, $displacement) = v = ")
            show(v)
            println()
            println("$wlabel = w = ")
            show(w)
            println()
            println("uv dist = $uv,  vw dist = $vw,  uw dist = $uw")
        end
    end
    println("""Converged to correct target: $converge_count
        Failed to converge: $failure_count
        Converged to wrong target: $wrong_target_count""")
end


In [ ]:
### NEAREST NEIGHBORS

# Given a vector label, find its nearest neighbor among all
# stored patterns, and report the hamming distance.

function nearest_neighbor(u::BitVector)
    best_label = ""
    best_distance = d
    for (l, v) in IDX
        dist = hamming_distance(u, v)
        if dist < best_distance
            best_label = l
            best_distance = dist
        end
    end
    return (best_label, best_distance)
end


function nearest_neighbor(label::String)
    haskey(IDX, label) || error("Item $label not indexed.")
    best_label = ""
    best_distance = d
    u = IDX[label]
    for (l, v) in IDX
        l == label && continue
        dist = hamming_distance(u, v)
        if dist < best_distance
            best_label = l
            best_distance = dist
        end
    end
    return (best_label, best_distance)
end


# Among all stored patterns, find the pair with the smallest
# Hamming distance

function nearest_neighbor_pair()
    best_left = ""
    best_right = ""
    best_distance = d
    ks = collect(keys(IDX))
    vs = collect(values(IDX))
    for i in 1:length(ks) - 1
        for j in i+1:length(ks)
            dist = hamming_distance(vs[i], vs[j])
            if dist < best_distance
                best_left = ks[i]
                best_right = ks[j]
                best_distance = dist
            end
        end
    end
    return (best_left, best_right, best_distance)
end


# For any two vectors (whether of not they are actually stored
# in the SDM), count the hard locations that would be common to
# their two access circles.

function hard_locations_in_common(u::BitVector, v::BitVector)
    u_indices = Vector{Int64}()
    v_indices = Vector{Int64}()
    for i in 1:length(SDM)
        if hamming_distance(u, SDM[i].address) <= r
            push!(u_indices, i)
        end
        if hamming_distance(v, SDM[i].address) <= r
            push!(v_indices, i)
        end
    end
    uv_indices = intersect(u_indices, v_indices)
    return (length(uv_indices), uv_indices)
end


# Probing the SDM with random vectors, count how many converge
# on a stored vector, how many converge on some other fixed point,
# and how many fail to converge within `limit` repetitions of the
# recursive recall process.

function fate_of_random_vectors(reps, limit=30)
    stored_item = 0
    other_fixed_point = 0
    did_not_converge = 0
    for i in 1:reps
        probe = make_rand_bitvec()
        fate = SDMsearch(probe, limit)
        if fate == zeros_bitvec
            did_not_converge += 1
        elseif IDXlookup(fate) == ""
            other_fixed_point += 1
        else
            stored_item += 1
        end
    end
    println("""
        stored items: $stored_item
        other fixed points: $other_fixed_point
        did not converge: $did_not_converge
    """)
end


### REINFORCEMENT THROUGH REPETITION

A well-known feature of human memory is that rehearsal improves recall. Repetition is how we memorize a poem or a list of state capitals. Something analogous happens in the SDM. Storing the same vector twice makes it easier to recall: The radius-of-convergence increases, so that the target pattern is accessible from a larger volume of the 1,000-dimensional hypercube.




In [ ]:

# Store the vector associated with `label` in the SDM `n` times.

function SDMstore(label::String, n::Int)
    for i in 1:n
        SDMstore(label)
    end
end




### DECIMATION AND RELIABILITY

One of the sterling qualities of distributed memory is robustness, the ability to retain information even when elements of the hardware fail. The `decimateSDM` procedure allows for testing this property. (Try running `convergence_fraction` before and after wiping out half the hard locations in the memory.) 

In [ ]:

# Zero out the content of a register, such as the register field
# of a hard location.

function reset_register!(reg)
    for i in 1:length(reg)
        reg[i] = 0
    end
end


# Zero the content registers of k randomly chosen hard locations
# in the SDM

function decimateSDM(k)
    target_indices = sample(collect(1:N), k, replace=false)
    for t in target_indices
        reset_register!(SDM[t].register)
    end
end



### BINDING AND BUNDLING

In [ ]:
### BINDING TWO VECTORS

# The xor operation has another vital role as well: It projects a random
# binary vector into another region of the d-dimensional space, preserving
# distance relationships. That is, if hamming_distance(u, v) == s, then also
# hamming_distance(xor.(u, c), xor.(v, c)) == s, when we project both u and v
# by c. Kanerva calls this projection operator "bind".

# Note that binding is commutative and associative. If SDMbind(u, v) == c, then
# SDMbind(v, u) == c, and SDMbind(u, c) == v and SDMbind(v, c) == u. Thus given any
# two of these vectors you can reconstruct the third.

# Also note that the dot in `xor.(u, v)` is Julia's "broadcast" notation: xor
# is applied elementwise to all pairs of bits in the two vectors.

SDMbind(u::BitVector, v::BitVector) = xor.(u, v)

SDMbind(ulabel::String, vlabel::String) = xor.(IDX[ulabel], IDX[vlabel])

SDMshowbinding(ulabel::String, vlabel::String) = IDXlookup(SDMfetch(SDMbind(ulabel, vlabel)))



In [ ]:
### BUNDLING

# A second key operation in Kanerva's algebra of high-dimensional vectors
# is "bundling" (also called "merging"). It is a componentwise sum or average,
# defined by a majority-rule algorithm.

# The bundling of just two vectors is particularly simple: At each bit position i, if
# u[i] and v[i] have the same value, then we choose that value in the result vector. 
# If u[i] and v[i] differ, the result is chosen at random.

function SDMbundle(u::BitVector, v::BitVector)
    (length(u) == length(v)) || throw(DimensionMismatch("Vectors u and v must have the same length."))
    result = BitVector(d)
    for i in 1:d
        result[i] = (u[i] == v[i]) ? u[i] : rand(Bool)
    end
    result
end


# For bundling more than two vectors, we could simply apply the method above
# repeatedly, perhaps in a binary-tree arrangement. But there's a better way -- 
# better both for speed and for minimizing information loss. We take any number of 
# input vectors "v..." or "v::VarArg", then do a signed count at each bit position,
# +1 for a 1 bit and -1 for a 0 bit. The output vector has a 1 (or true) wherever 
# the sum is positive, a 0 (or false) wherever the sum is negative, and a random 
# bit where the sum is 0.

function SDMbundle(v::Vararg{BitVector})
    accum = zeros(Int64, d)
    result = BitVector(d)
    for vec in v
        for i in 1:d
            accum[i] += vec[i] ? 1 : -1
        end
    end
    for j in 1:d
        if accum[j] > 0
            result[j] = true
        elseif accum[j] < 0
            result[j] = false
        else
            result[j] = rand(Bool)
        end
    end
    result
end
